In [105]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import talib as ta2
import plotly.express as px
from datetime import date
import plotly.graph_objects as go
from yahooquery import Ticker

# yahooquery wokrs better than yfinance
# yahooquery project documentation: https://pypi.org/project/yahooquery/
# yfinance project documentation: https://pypi.org/project/yfinance/
# pandas_ta project documentation: https://twopirllc.github.io/pandas-ta/


In [106]:
# Define functions to retrieve market data and create graphs
def get_ticker(ticker):
    try:
        ticker = Ticker(ticker)
        return ticker
    except:
        print('Error: Ticker not found')
        return None

def get_price_hist(ticker, period, interval):
    hist = ticker.history(period=period, interval=interval)
    hist = hist.reset_index()
    # hist['date'] = pd.to_datetime(hist['date'])
    # hist['date'] = hist['date'].dt.strftime('%Y-%m-%d')
    return hist



def get_financials(df, col_name, metric_name):
    metric = df.loc[:, ['asOfDate', col_name]]
    metric_df = pd.DataFrame(metric).reset_index()
    metric_df.columns = ['Symbol', 'Year', metric_name]

    return metric_df


In [208]:
# Simple Plot for fundamental financial data
def line_chart(df, x, y, title):
    fig = px.line(df, x=x, y=y, template='simple_white',
                        title='<b>{} {}</b>'.format(name, title))

    return fig

def bar_chart(df, x, y, title):
    fig = px.bar(df, x=x, y=y, template='simple_white',
                        title='<b>{} {}</b>'.format(name, title))

    return fig



In [202]:
# Get today's date
today = date.today()

# Enter company ticker. 
ticker_string = input('Enter company ticker: ')

# trim user input string
ticker_string = str(ticker_string.lower()).strip()

# today = date.today()

# Pass the user input to the yahooquery library
ticker = get_ticker(ticker_string)

# Extract Fundamental Data from Yahoo Finance

In [203]:
# check if ticker is a ETF or stock
security_type = ticker.price[ticker_string]['quoteType']
if security_type == 'ETF':
    stock = False
else:
    stock = True




if stock:
    name = ticker.price[ticker_string]['shortName']
    sector = ticker.summary_profile[ticker_string]['sector']
    industry = ticker.summary_profile[ticker_string]['industry']
    holdings = 'NA'
    summary = ticker.summary_profile[ticker_string]['longBusinessSummary']
        # employees = ticker.summary_profile[ticker_string]['fullTimeEmployees']
        # country = ticker.summary_profile[ticker_string]['country']
        # city = ticker.summary_profile[ticker_string]['city']
        # website = ticker.summary_profile[ticker_string]['website']

    # create a dictionary with the required keys and values
    fundamental_data = {'Name': [name], 'Sector': [sector], 'Industry': [industry], 'Summary': [summary]}

    # pass the dictionary to the pandas DataFrame constructor
    fundamental_df = pd.DataFrame(fundamental_data)

    # display fundamental data
    display(fundamental_df)
else:
    pass

,Name,Sector,Industry,Summary
0,"MYR Group, Inc.",Industrials,Engineering & Construction,"MYR Group Inc., through its subsidiaries, prov..."


In [204]:

if stock:
    # This is a summary of the company's financial data
    valuation_df = ticker.valuation_measures
    # Restrict valuation_df to periodType = 'TTM'
    valuation_df = valuation_df[valuation_df['periodType'] == 'TTM']
    financial_dict = ticker.financial_data[ticker_string]
    financial_df = pd.DataFrame.from_dict(financial_dict, orient='index').reset_index().rename(columns={'index': 'Metric', 0: 'Value'})

    # # More detailed financial data for the company
    
    # income_df = ticker.income_statement()
    # balance_df = ticker.balance_sheet()
    # cash_df = ticker.cash_flow()
   

else:
    holdings = ticker.fund_holding_info[ticker_string]['holdings']
    top_holdings_df = pd.DataFrame(holdings)
    # Rename the columns
    top_holdings_df.columns = ['Symbol','Name', 'Percent']
    # Express the holdings column as a percentage
    top_holdings_df['Percent'] = top_holdings_df['Percent'].apply(lambda x: x * 100)
    # Add a column to the dataframe with today's date
    top_holdings_df['Date'] = today
    # # Display the dataframe
    # display(top_holdings_df)



# Goal
================
* Iterate through a list of tickers
* Retrieve historical data from Yahoo Finance
* Determine range bound patterns in the data such as channels and triangles
* Check which tickers are in a Channel, Triangle or Wedge Patterns
* Determine likely breakout candidates
* Determine likely breakdown candidates
* Determine likely range bound candidates

Example Tickers:
================
* MYRG
* HAE
* LVMUY




Technical Indicators Definitions and Examples
================
https://library.tradingtechnologies.com/trade/chrt-technical-indicators.html

Chart Patterns
================
https://www.incrediblecharts.com/technical/chart_patterns.php


#  Average directional index (ADX)
ADX is used to quantify **trend strength**.

| ADX Value | Trend Strength           |
|-----------|-------------------------|
| 0-25      | Absent or Weak Trend     |
| 25-50     | Strong Trend             |
| 50-75     | Very Strong Trend        |
| 75-100    | Extremely Strong Trend   |


In [205]:
# Create a Strategy
mvg_avg = ta.Strategy(name="Dual_emas", ta=[{"kind": "ema", "length": 50}, {"kind": "ema", "length": 100}, {"kind": "sma", "length": 200}])

In [219]:

def plot_candlestick(df, slider=False, ma=False):
    # Create figure
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                                         open=df['open'],
                                         high=df['high'],
                                         low=df['low'],
                                         close=df['close'])]                                   
                    )
    # Add moving averages
    if ma:
        # Add the strategy to the DataFrame
        df.ta.strategy(mvg_avg)
        # Add EMA and SMA lines
        fig.add_trace(go.Scatter(x=df['date'], y=df['EMA_50'], line=dict(color='pink', width=1), name='EMA 50'))
        fig.add_trace(go.Scatter(x=df['date'], y=df['EMA_100'], line=dict(color='purple', width=1), name='EMA 100'))
        fig.add_trace(go.Scatter(x=df['date'], y=df['SMA_200'], line=dict(color='red', width=2), name='SMA 200'))
    else:
        pass
    # Change figure size
    fig.update_layout(
        autosize=True,
        width=1000,
        height=600
        )
    

    # Add title and xaxis
    fig.update_layout(
        title=f'{ticker_string.upper()} Stock Price',
        xaxis_rangeslider_visible=slider,
        xaxis_title='Date',
        yaxis_title='Price',
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    return fig


In [221]:
# Define chart options
period_option = '10y'
interval_option = '1wk' # interval options: 1d, 1wk, 1mo

# Get price history
stock_df = get_price_hist(ticker=ticker, period=period_option, interval=interval_option)



In [222]:
# cross = ta.cross(stock_df['EMA_50'], stock_df['EMA_100'], above=True)

# # check where the cross happened
# cross[cross == True]

In [223]:
candlestic = plot_candlestick(stock_df, ma=False)

candlestic.show()

In [ ]:
# balance_df.reset_index(inplace=True)
# # Convert asOfDate column to year only
# balance_df['asOfDate'] = balance_df['asOfDate'].dt.year
# # Rename asOfDate column to year
# balance_df.rename(columns={'asOfDate': 'Year'}, inplace=True) 

# rev_df = get_financials(df=income_df, col_name='TotalRevenue', metric_name='Total Revenue')
# # Convert Year column to year only
# rev_df['Year'] = rev_df['Year'].dt.year 

# marketcap_df = get_financials(df=valuation_df, col_name='MarketCap', metric_name='Market Cap') # This is updated quarterly

# ebitda_df = get_financials(df=income_df, col_name='NormalizedEBITDA', metric_name='EBITDA')
# # Convert Year column to year only
# ebitda_df['Year'] = ebitda_df['Year'].dt.year


# rev_fig = bar_chart(df=rev_df, x='Year', y='Total Revenue', title='Total Revenue USD')
# rev_fig.show()

# marketcap_fig = line_chart(df=marketcap_df, x='Year', y='Market Cap', title='Market Cap USD')
# marketcap_fig.show()


# ebitda_fig = bar_chart(df=ebitda_df, x='Year', y='EBITDA', title='EBITDA USD')
# ebitda_fig.show()